In [5]:
import pandas as pd
from numpy import random
from faker import Faker

In [19]:
trans = pd.read_json('../00_data/transition_matrix.json')
trans

,checkout,dairy,drinks,fruit,spices
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.393033,0.000000,0.222483,0.189357,0.195127
drinks,0.537260,0.027145,0.000000,0.218950,0.216645
fruit,0.500195,0.237993,0.136080,0.000000,0.125732
spices,0.251998,0.323122,0.272776,0.152104,0.000000


In [7]:
trans.loc['dairy', ]

checkout    0.393033
dairy       0.000000
drinks      0.222483
fruit       0.189357
spices      0.195127
Name: dairy, dtype: float64

In [8]:
initial_condition = pd.read_json('../00_data/initial_condition.json',typ='Series')
initial_condition

checkout    0.000000
dairy       0.312085
drinks      0.159363
fruit       0.304117
spices      0.224436
dtype: float64

In [9]:
random.choice(initial_condition.index.values, p = initial_condition)

'spices'

In [140]:


class Customer:
    """
    a single customer that moves through the supermarket
    in a MCMC simulation
    """
    ...
    def __init__(self, initial_state_probabilities, transition_matrix, deltat, budget=100):
        fk = Faker()
        
        self.name =  fk.name()
        self.budget = budget
        self.initial_state_probabilities = initial_state_probabilities
        self.state = random.choice(initial_state_probabilities.index.values, p = initial_state_probabilities)
        self.transition_matrix = transition_matrix
        self.steps = [self.state]
        self.deltat=deltat
        
    
    def __repr__(self):
        return f'<Customer {self.name} in {self.state}>'

    def next_state(self):
        '''
        Propagates the customer to the next state.
        Returns nothing.
        '''
        choices = self.transition_matrix.index.values
        probs = self.transition_matrix.loc[self.state]
        # print(choices, probs)
        self.state = random.choice(choices, p = probs)
        self.steps.append(self.state)

    def is_active(self):
        """Returns True if the customer has not reached the checkout yet."""
        if self.state!='checkout':
            return True
        else:
            return False

    def sim(self):
        self.next_state()
        while self.is_active():
            self.next_state()

    def get_behavior(self):
        assert isinstance(self.deltat, pd._libs.tslibs.timedeltas.Timedelta), 'Need a time delta'
        if not self.is_active():
            timedf = pd.DataFrame({'name' : self.name, 'steps' : self.steps, 'time' : [self.deltat * v for v in range(len(self.steps))]})
            return timedf
        else:
            return None

        



In [167]:
c1 = Customer(transition_matrix=trans,
        initial_state_probabilities=initial_condition,
        deltat=pd.Timedelta(5, 'minutes'))
print(c1)

c1.sim()
print(c1.steps)
print(c1)
c1.get_behavior()

<Customer Jeremy Singleton in spices>
['spices', 'checkout']
<Customer Jeremy Singleton in checkout>


,name,steps,time
0,Jeremy Singleton,spices,0 days 00:00:00
1,Jeremy Singleton,checkout,0 days 00:05:00


next steps:

 - add time delta for each step (by distance between sections? use dot product to calculate?)
 - simulate for many

In [143]:
n_customers = 1000
# customers = [Customer(transition_matrix=trans, initial_state_probabilities=initial_condition) for i in range(n_customers)]
custs = [None] * n_customers

for i in range(n_customers):
    custs[i] = Customer(transition_matrix=trans, initial_state_probabilities=initial_condition,deltat= pd.Timedelta(5, 'minutes'))



In [144]:
customer_steps = [None] * n_customers

for idx, c in enumerate(custs):
    c.sim()
    customer_steps[idx] = c.steps

In [168]:
for j in range(10):
    print(custs[j], custs[j].steps)

<Customer Heidi Bishop in checkout> ['dairy', 'fruit', 'drinks', 'fruit', 'drinks', 'spices', 'fruit', 'drinks', 'checkout']
<Customer Dennis Jimenez in checkout> ['fruit', 'checkout']
<Customer Beverly Orr in checkout> ['dairy', 'checkout']
<Customer Samuel Washington in checkout> ['fruit', 'checkout']
<Customer Christina Williams in checkout> ['drinks', 'spices', 'drinks', 'checkout']
<Customer Daniel Roberts in checkout> ['dairy', 'fruit', 'checkout']
<Customer Stephanie Chen in checkout> ['dairy', 'checkout']
<Customer Jose Gomez in checkout> ['dairy', 'drinks', 'checkout']
<Customer Mary Brown in checkout> ['dairy', 'drinks', 'checkout']
<Customer Kristin Ballard in checkout> ['fruit', 'checkout']


In [169]:
pd.to_datetime("2022-07-06")

Timestamp('2022-07-06 00:00:00')

## With supermarket class

In [196]:
import functools

In [239]:


class supermarket:
    
    """
    a supermarket that operates for one/x days
    holding mc customers
    """
    ...
    def __init__(self, opening, closing, entry_rate, super_deltat, customer_deltat, transition_matrix, initial_probs):
        fk = Faker()
        
        self.opening =  opening
        self.closing = closing
        self.entry_rate = entry_rate
        self.sdt = super_deltat
        self.cdt = customer_deltat 
        self.transition_matrix = transition_matrix
        self.initial_probs = initial_probs
        self.customers = None

        self.customer_log = None

        self.n_steps = int(round((self.closing - self.opening) / self.sdt))

    def make_logs(self):
        self.customer_log = pd.concat(map(pd.concat, self.operations))
        return self.customer_log


    def get_customers(self):

        # distribute n customers across time step
        cust = [None] * self.entry_rate

        for i in range(self.entry_rate):
            cust[i] = Customer(
                transition_matrix=self.transition_matrix,
                initial_state_probabilities=self.initial_probs,
                deltat= self.cdt)

        self.customers = {'customers' : cust,
         'relative_entry_time' : random.uniform(size=self.entry_rate) * self.sdt}

        return self.customers


    
    def operate_market(self):

        dt_list = [None] * self.n_steps

        for step in range(self.n_steps):

            customers = self.get_customers()

            step_start = step * self.sdt + self.opening

            
            # customer_time = list(map(lambda x: pd.Timedelta(x, "minutes") + step_start, customers['relative_entry_time']))
            customer_time = list(map(lambda x: x + step_start, customers['relative_entry_time']))



            customer_activity_list = [None] * len(customers['customers'])

            for idx, cs in enumerate(customers['customers']):

                cs.sim()
                cdf = cs.get_behavior()
                # print(cdf)
                # print(cdf['time'])
                # print(customer_time[idx])
                cdf['time'] = cdf['time'].apply(lambda x: x  +  customer_time[idx])

                customer_activity_list[idx] = cdf

            dt_list[step] = customer_activity_list

        self.operations = dt_list

        return self.operations
     
    # entry rate and super delta t must have same temporal resoltion, use hourly

    # cycle from opening to closing by super_delta t

    # for every step, add n customers at a random entry time

    # execute sim calculate sim time + entry time

    # at last step, check if any customers extend past closing time.
    # drop all steps at or past closing time, and add a checkout at closing time instead

In [240]:
tesco = supermarket(
    opening = pd.to_datetime("2022-07-06 07:00:00"),
    closing=pd.to_datetime("2022-07-06 22:00:00"),
    super_deltat=pd.Timedelta(1, 'H'),
    customer_deltat=pd.Timedelta(5, 'minutes'),
    entry_rate=10, # per hour,
    transition_matrix=trans,
    initial_probs=initial_condition
    )

In [241]:
tesco.get_customers()

{'customers': [<Customer Amanda Bright in drinks>,
  <Customer Mark Nolan in dairy>,
  <Customer Megan Holland in dairy>,
  <Customer Heidi Gonzalez in fruit>,
  <Customer Jared Colon in spices>,
  <Customer Gregory Potts in spices>,
  <Customer Brenda Rivera in dairy>,
  <Customer Alexa Massey in drinks>,
  <Customer David Galvan in drinks>,
  <Customer Carlos Mcdonald in fruit>],
 'relative_entry_time': array([ 838190324572,  290651650574, 3294344158102, 2663144192655,
         263426386364, 1948024860413, 1704985715699, 1045407725766,
         793965410783, 2998589965744], dtype='timedelta64[ns]')}

In [242]:
tesco.operate_market()

[[            name     steps                          time
  0  Emily Higgins     fruit 2022-07-06 07:18:24.975739248
  1  Emily Higgins  checkout 2022-07-06 07:23:24.975739248,
            name     steps                          time
  0  Eric Barker     fruit 2022-07-06 07:25:36.366226028
  1  Eric Barker     dairy 2022-07-06 07:30:36.366226028
  2  Eric Barker    spices 2022-07-06 07:35:36.366226028
  3  Eric Barker    drinks 2022-07-06 07:40:36.366226028
  4  Eric Barker  checkout 2022-07-06 07:45:36.366226028,
              name     steps                          time
  0  Angela Thomas     dairy 2022-07-06 07:54:07.251517134
  1  Angela Thomas     fruit 2022-07-06 07:59:07.251517134
  2  Angela Thomas     dairy 2022-07-06 08:04:07.251517134
  3  Angela Thomas     fruit 2022-07-06 08:09:07.251517134
  4  Angela Thomas  checkout 2022-07-06 08:14:07.251517134,
            name     steps                          time
  0  Sandra Park    drinks 2022-07-06 07:35:51.754506630
  1  Sandr

In [243]:
tesco.make_logs()

,name,steps,time
0,Emily Higgins,fruit,2022-07-06 07:18:24.975739248
1,Emily Higgins,checkout,2022-07-06 07:23:24.975739248
0,Eric Barker,fruit,2022-07-06 07:25:36.366226028
1,Eric Barker,dairy,2022-07-06 07:30:36.366226028
2,Eric Barker,spices,2022-07-06 07:35:36.366226028
...,...,...,...
0,Thomas Ferguson,drinks,2022-07-06 21:26:14.108721038
1,Thomas Ferguson,fruit,2022-07-06 21:31:14.108721038
2,Thomas Ferguson,drinks,2022-07-06 21:36:14.108721038
3,Thomas Ferguson,fruit,2022-07-06 21:41:14.108721038


In [244]:
tesco.customer_log

,name,steps,time
0,Emily Higgins,fruit,2022-07-06 07:18:24.975739248
1,Emily Higgins,checkout,2022-07-06 07:23:24.975739248
0,Eric Barker,fruit,2022-07-06 07:25:36.366226028
1,Eric Barker,dairy,2022-07-06 07:30:36.366226028
2,Eric Barker,spices,2022-07-06 07:35:36.366226028
...,...,...,...
0,Thomas Ferguson,drinks,2022-07-06 21:26:14.108721038
1,Thomas Ferguson,fruit,2022-07-06 21:31:14.108721038
2,Thomas Ferguson,drinks,2022-07-06 21:36:14.108721038
3,Thomas Ferguson,fruit,2022-07-06 21:41:14.108721038
